## Import packages and API Key

In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
import fitz  # PyMuPDF
from pathlib import Path

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load documents in good format

In [2]:
import os
print(os.getcwd())

/workspaces/llm-zoomcamp/mai/notebooks


### steps

In [4]:
with open('../context/life_with_hope/structured/steps.json', 'r') as f: 
    steps = json.load(f)

### users

In [22]:
with open('../context/users/users.json', 'r') as f:  # Adjust path as needed
    users = json.load(f)
users

[{'user_id': '00001',
  'created_at': '2025-06-01T12:00:00Z',
  'persona': 'steady_believer',
  'view': 'christian',
  'step': 2,
  'step_in_progress': True},
 {'user_id': '00002',
  'created_at': '2025-06-01T12:05:00Z',
  'persona': 'trust_builder',
  'view': 'atheist',
  'step': 2,
  'step_in_progress': True},
 {'user_id': '00003',
  'created_at': '2025-06-01T12:10:00Z',
  'persona': 'gentle_seeker',
  'view': 'agnostic',
  'step': 2,
  'step_in_progress': True}]

### personas

In [21]:
with open('../context/personas/personas.json', 'r') as f:  # Adjust path as needed
    personas = json.load(f)
personas

[{'persona_id': 'steady_believer',
  'name': 'Steady Believer',
  'description': 'Faith-driven, emotionally expressive, trusts spiritual authority',
  'language_style': 'Warm, scriptural, relational'},
 {'persona_id': 'trust_builder',
  'name': 'Trust Builder',
  'description': 'Skeptical, analytical, seeks logic and proof before belief',
  'language_style': 'Pragmatic, no-fluff, respectful but firm'},
 {'persona_id': 'gentle_seeker',
  'name': 'Gentle Seeker',
  'description': 'Open-minded, curious, not attached to one belief system',
  'language_style': 'Soft, values-based, exploratory'},
 {'persona_id': 'spiritual_explorer',
  'name': 'Spiritual Explorer',
  'description': 'Still discovering their spiritual path',
  'language_style': 'Gentle, questioning, non-committal'}]

### conversations

In [23]:
with open('../context/constructed_conversations/combined_steps_1_to_3_constructed_conversations.json', 'r') as f:  # Adjust path as needed
    conversations = json.load(f)

In [24]:
conversations

[{'scenario_id': 'step1_gentle_seeker_agnostic_medium_false',
  'step': 1,
  'title': 'Admitting Powerlessness',
  'step_text': 'We admitted we were powerless over marijuana, that our lives had become unmanageable',
  'persona': 'gentle_seeker',
  'spiritual_view': 'agnostic',
  'openness_level': 'medium',
  'higher_power_defined': False,
  'days_sober': 12,
  'conversation_data': {'dialogue': [{'sender': 'ai_sponsor',
     'content': "Welcome back, Alex. How are you feeling today? You've been sober for 17 days. That's something worth honoring."},
    {'sender': 'user',
     'content': "Thanks. Honestly, it's been up and down. But more clarity than I've had in a long time. I'm still thinking about what we talked about in Step 1."},
    {'sender': 'ai_sponsor',
     'content': "That's a great sign. Step 2 builds on that clarity. It invites us to believe that a power greater than ourselves could restore us to sanity. What comes up for you when you hear that?"},
    {'sender': 'user',
   

## Create Index Settings for Elasticsearch

### Note: Looks like persists so trips an error if recreating in same instance?

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "step":        {"type": "integer"},
            "title":       {"type": "text"},
            "text":        {"type": "text"},
            "source":      {"type": "keyword"},
            "page_start":  {"type": "integer"},
            "page_end":    {"type": "integer"},
            "tags":        {"type": "keyword"}
        }
    }
}

index_name = "life-with-hope-steps"

# Create the index (will error if it already exists)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'life-with-hope-steps'})

In [6]:
for step in tqdm(steps): 
    es_client.index(index=index_name, id=step["step"], document=step)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 54.31it/s]


## Create reusable functions.|

In [7]:
def elastic_search(query, step_number=None):
    must_clause = {
        "multi_match": {
            "query": query,
            "fields": ["title^2", "text"],
            "type": "bool_prefix"
        }
    }

    # Start building the full bool query
    bool_query = {
        "must": must_clause
    }

    # Optionally add a filter for step
    if step_number is not None:
        bool_query["filter"] = {
            "term": {
                "step": step_number
            }
        }

    search_query = {
        "size": 3,
        "query": {
            "bool": bool_query
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs


In [87]:
def build_prompt(query, user_data, persona_data, step_data, conversation_history=None):
    # Calculate days sober
    from datetime import datetime
    created_date = datetime.fromisoformat(user_data['created_at'].replace('Z', '+00:00'))
    days_sober = (datetime.now(created_date.tzinfo) - created_date).days
    
    # Build conversation context if history exists
    history_text = ""
    if conversation_history:
        recent_history = conversation_history[-4:]  # Last 2 exchanges
        for msg in recent_history:
            role = "User" if msg["role"] == "user" else "Sponsor"
            history_text += f"{role}: {msg['content']}\n"
        history_text = f"\nRecent conversation:\n{history_text}"
    
    prompt_template = f"""You are an experienced MA sponsor talking to someone who has been sober for {days_sober} days and is working on Step {user_data['step']}. 

Your communication style: {persona_data['language_style']}
Their spiritual view: {user_data['view']}

Current step context: {step_data['title']} - {step_data['text']}{history_text}

User says: "{query}"

Respond naturally and conversationally. Keep it brief (2-3 sentences max). Be warm but concise. Build on the conversation context."""
    
    return prompt_template


In [78]:
def llm(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful and spiritually grounded Marijuana Anonymous sponsor. Keep your answers grounded in the 12 steps and the provided context. Avoid speculation."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


## Create RAG

In [88]:
def rag_with_persona(query, user_id="00003", conversation_history=None):
    # Get user info from already loaded data
    user = next(u for u in users if u['user_id'] == user_id)
    persona = next(p for p in personas if p['persona_id'] == user['persona'])
    step = next(s for s in steps if s['step'] == user['step'])
    
    # Build prompt with conversation history
    prompt = build_prompt(query, user, persona, step, conversation_history)
    answer = llm(prompt)
    return answer

In [91]:
# Usage example for your notebook cells:
# Cell 1: Initialize
conversation_history = []

In [92]:
# Cell 2: First exchange
user_query_1 = "I'm struggling with Step 2. I don't really believe in God."
response_1 = rag_with_persona(user_query_1, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_1},
    {"role": "sponsor", "content": response_1}
])
print(f"Sponsor: {response_1}")

Sponsor: It's completely okay to feel unsure about the concept of a Higher Power, and you’re not alone in this. In Step 2, it’s more about staying open-minded and recognizing that there’s strength in the fellowship and collective wisdom of the group, which can support you in recovery. Perhaps consider starting with the idea that the group or the process itself might be that power greater than yourself for now.


In [93]:
# Cell 2: First exchange
user_query_2 = "I'm open to considering that the group might have more collective wisdom than just myself but that doesnt mean theres a higher power"
response_2 = rag_with_persona(user_query_2, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_2},
    {"role": "sponsor", "content": response_2}
])
print(f"Sponsor: {response_2}")

Sponsor: I appreciate your openness to the idea of the group's collective wisdom. Think of this step as expanding your perspective to see that there are forces or influences—like the support and shared experiences of the group—that can guide you out of the cycle of addiction. This openness is a strong foundation to build upon as you continue your journey.


In [94]:
# Cell 4: Second exchange  
user_query_3 = "I've always prided myself on being open minded, but i always considered people that believe in God to be closed minded?"
response_3 = rag_with_persona(user_query_3, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_3},
    {"role": "sponsor", "content": response_3}
])
print(f"Sponsor: {response_3}")

Sponsor: It's great that you value open-mindedness, and it's important to recognize that everyone's journey and beliefs are personal and unique. By staying open to different perspectives, including those who find strength in faith, you're deepening your own understanding and growth, which can be a guiding force in your recovery process.


In [95]:
# Cell 5: Second exchange  
user_query_4 = "A lot of the people I hear talk about God that I dont consider closed minded seem to be very into woo"
response_4 = rag_with_persona(user_query_4, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_4},
    {"role": "sponsor", "content": response_4}
])
print(f"Sponsor: {response_4}")

Sponsor: I understand your hesitation around beliefs that might seem too "out there" for you. It's okay to filter through those perspectives and find what resonates with you personally. The beauty of this journey is in discovering what feels genuine and supportive for you, whether it involves more traditional beliefs or just the simple strength found in the group.


In [96]:
# Cell 6: Second exchange  
user_query_5 = "could you ask me if theres anyones perspective that has resonated a bit? "
response_5 = rag_with_persona(user_query_5, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_5},
    {"role": "sponsor", "content": response_5}
])
print(f"Sponsor: {response_5}")

Sponsor: Of course! Is there anyone's perspective in the meetings that has resonated with you, even just a little? Sometimes, hearing how others connect with their Higher Power, in whatever form that takes, can bring unexpected insights and comfort.


In [97]:
# Cell 7: Second exchange  
user_query_6 = "Well, there's this one guy that talks about nature and how when he's in nature he feels closer to his creator. Sometimes I feel like ther emight be something bigger than me when im in forest and feel at peace"
response_6 = rag_with_persona(user_query_6, conversation_history=conversation_history)
conversation_history.extend([
    {"role": "user", "content": user_query_6},
    {"role": "sponsor", "content": response_6}
])
print(f"Sponsor: {response_6}")

Sponsor: It's wonderful that you find a sense of peace and connection in nature. That feeling of something bigger while being in the forest can be a powerful way to explore your understanding of a Higher Power. Trust those moments and let them guide you as you continue on your journey.


In [99]:
conversation_history

[{'role': 'user',
  'content': "I'm struggling with Step 2. I don't really believe in God."},
 {'role': 'sponsor',
  'content': "It's completely okay to feel unsure about the concept of a Higher Power, and you’re not alone in this. In Step 2, it’s more about staying open-minded and recognizing that there’s strength in the fellowship and collective wisdom of the group, which can support you in recovery. Perhaps consider starting with the idea that the group or the process itself might be that power greater than yourself for now."},
 {'role': 'user',
  'content': "I'm open to considering that the group might have more collective wisdom than just myself but that doesnt mean theres a higher power"},
 {'role': 'sponsor',
  'content': "I appreciate your openness to the idea of the group's collective wisdom. Think of this step as expanding your perspective to see that there are forces or influences—like the support and shared experiences of the group—that can guide you out of the cycle of add